In [18]:
!pip install pyspark findspark pandas

     |████████████████████████████████| 9.5 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 86.5 MB/s eta 0:00:01


In [19]:
import pandas as pd

In [8]:
import findspark
findspark.init()

import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [9]:
from pyspark.sql import functions as F

simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [10]:
df.groupBy("department").sum("salary").show(truncate=False)

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|Sales     |257000     |
|Finance   |351000     |
|Marketing |171000     |
+----------+-----------+



In [11]:
df.groupBy("department").count().show(truncate=False)

+----------+-----+
|department|count|
+----------+-----+
|Sales     |3    |
|Finance   |4    |
|Marketing |2    |
+----------+-----+



In [12]:
df.groupBy("department","state") \
    .sum("salary","bonus") \
   .show(truncate=False)

+----------+-----+-----------+----------+
|department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
|Finance   |NY   |162000     |34000     |
|Marketing |NY   |91000      |21000     |
|Sales     |CA   |81000      |23000     |
|Marketing |CA   |80000      |18000     |
|Finance   |CA   |189000     |47000     |
|Sales     |NY   |176000     |30000     |
+----------+-----+-----------+----------+



In [23]:

df.groupBy("department") \
    .agg(
  F.sum("salary").alias("sum_salary"), \
         F.avg("salary").alias("avg_salary"), \
         F.sum("bonus").alias("sum_bonus"), \
         F.max("bonus").alias("max_bonus") \
     ) \
    .show(truncate=False)

+----------+----------+-----------------+---------+---------+
|department|sum_salary|avg_salary       |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
|Sales     |257000    |85666.66666666667|53000    |23000    |
|Finance   |351000    |87750.0          |81000    |24000    |
|Marketing |171000    |85500.0          |39000    |21000    |
+----------+----------+-----------------+---------+---------+



In [20]:
df.groupBy("department") \
    .agg(
  F.sum("salary").alias("sum_salary"), \
         F.avg("salary").alias("avg_salary"), \
         F.sum("bonus").alias("sum_bonus"), \
         F.max("bonus").alias("max_bonus") \
     ) \
    .toPandas()

,department,sum_salary,avg_salary,sum_bonus,max_bonus
0,Sales,257000,85666.666667,53000,23000
1,Finance,351000,87750.000000,81000,24000
2,Marketing,171000,85500.000000,39000,21000


In [32]:
df.write.mode('overwrite').parquet('./test_df.parquet')


In [27]:
!du -h -d 2 ./test_df.parquet/*

4.0K	./test_df.parquet/part-00000-79157ecc-18c1-4c4c-8d2a-2015effe1646-c000.snappy.parquet
4.0K	./test_df.parquet/part-00001-79157ecc-18c1-4c4c-8d2a-2015effe1646-c000.snappy.parquet
0	./test_df.parquet/_SUCCESS


In [28]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip

--2021-10-02 15:45:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z 100%[===================>] 198.65K   262KB/s    in 0.8s    

2021-10-02 15:45:25 (262 KB/s) - ‘smsspamcollection.zip’ saved [203415/203415]



In [29]:
!ls

 Seminar3.ipynb			   spark_environ.sh
 smsspamcollection.zip		   test_df.parquet
 spark-3.1.2-bin-hadoop2.7.tgz	  'test notebook.ipynb'
 spark-3.1.2-bin-hadoop2.7.tgz.1   venv


In [30]:
!unzip smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [31]:
src = (
    spark
    .read
    .option("sep", "\t")
    .csv("./SMSSpamCollection")
    .withColumnRenamed("_c0", "label")
    .withColumnRenamed("_c1", "message")
)

In [37]:
src.write.partitionBy('label').mode('overwrite').parquet('./test_df.parquet')


In [38]:
!du -h -d 2 ./test_df.parquet/*

256K	./test_df.parquet/label=ham
68K	./test_df.parquet/label=spam
0	./test_df.parquet/_SUCCESS


In [40]:
ham = spark.read.parquet('./test_df.parquet/label=ham')
ham.limit(10).toPandas()

,message
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,U dun say so early hor... U c already then say...
3,"Nah I don't think he goes to usf, he lives aro..."
4,Even my brother is not like to speak with me. ...
5,As per your request 'Melle Melle (Oru Minnamin...
6,I'm gonna be home soon and i don't want to tal...
7,I've been searching for the right words to tha...
8,I HAVE A DATE ON SUNDAY WITH WILL!!
9,Oh k...i'm watching here:)


In [44]:
src.repartition(10).write.partitionBy('label').mode('overwrite').parquet('./test_df.parquet')  # это не очень хорошее действи
# делать партишт бай перед записью большими данными
# а репартшишн - на более мелкое число партиций
!du -h -d 2 ./test_df.parquet/*

332K	./test_df.parquet/label=ham
152K	./test_df.parquet/label=spam
0	./test_df.parquet/_SUCCESS


In [ ]:
# df1.join().join().groupby().repartition() # - это хорошая вещь посл групбай

In [21]:
df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
      avg("salary").alias("avg_salary"), \
      sum("bonus").alias("sum_bonus"), \
      max("bonus").alias("max_bonus")) \
    .where(col("sum_bonus") >= 50000) \
    .show(truncate=False)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [0]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]

df = spark.createDataFrame(data = simpleData, schema = columns)

df.printSchema()
df.show(truncate=False)

root
-- employee_name: string (nullable = true)
-- department: string (nullable = true)
-- salary: long (nullable = true)

+-------------+----------+------+
employee_name|department|salary|
+-------------+----------+------+
James |Sales |3000 |
Michael |Sales |4600 |
Robert |Sales |4100 |
Maria |Finance |3000 |
James |Sales |3000 |
Scott |Finance |3300 |
Jen |Finance |3900 |
Jeff |Marketing |3000 |
Kumar |Marketing |2000 |
Saif |Sales |4100 |
+-------------+----------+------+

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec  = Window.partitionBy("department").orderBy("salary")

df.withColumn("row_number",row_number().over(windowSpec)) \
    .show(truncate=False)


+-------------+----------+------+----------+
employee_name|department|salary|row_number|
+-------------+----------+------+----------+
Maria |Finance |3000 |1 |
Scott |Finance |3300 |2 |
Jen |Finance |3900 |3 |
Kumar |Marketing |2000 |1 |
Jeff |Marketing |3000 |2 |
James |Sales |3000 |1 |
James |Sales |3000 |2 |
Robert |Sales |4100 |3 |
Saif |Sales |4100 |4 |
Michael |Sales |4600 |5 |
+-------------+----------+------+----------+

In [0]:
from pyspark.sql.functions import rank
df.withColumn("rank",rank().over(windowSpec)) \
    .show()

+-------------+----------+------+----+
employee_name|department|salary|rank|
+-------------+----------+------+----+
 Maria| Finance| 3000| 1|
 Scott| Finance| 3300| 2|
 Jen| Finance| 3900| 3|
 Kumar| Marketing| 2000| 1|
 Jeff| Marketing| 3000| 2|
 James| Sales| 3000| 1|
 James| Sales| 3000| 1|
 Robert| Sales| 4100| 3|
 Saif| Sales| 4100| 3|
 Michael| Sales| 4600| 5|
+-------------+----------+------+----+

In [0]:
from pyspark.sql.functions import dense_rank
df.withColumn("dense_rank",dense_rank().over(windowSpec)) \
    .show()

+-------------+----------+------+----------+
employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
 Maria| Finance| 3000| 1|
 Scott| Finance| 3300| 2|
 Jen| Finance| 3900| 3|
 Kumar| Marketing| 2000| 1|
 Jeff| Marketing| 3000| 2|
 James| Sales| 3000| 1|
 James| Sales| 3000| 1|
 Robert| Sales| 4100| 2|
 Saif| Sales| 4100| 2|
 Michael| Sales| 4600| 3|
+-------------+----------+------+----------+

In [0]:
from pyspark.sql.functions import percent_rank
df.withColumn("percent_rank",percent_rank().over(windowSpec)) \
    .show()

+-------------+----------+------+------------+
employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
 Maria| Finance| 3000| 0.0|
 Scott| Finance| 3300| 0.5|
 Jen| Finance| 3900| 1.0|
 Kumar| Marketing| 2000| 0.0|
 Jeff| Marketing| 3000| 1.0|
 James| Sales| 3000| 0.0|
 James| Sales| 3000| 0.0|
 Robert| Sales| 4100| 0.5|
 Saif| Sales| 4100| 0.5|
 Michael| Sales| 4600| 1.0|
+-------------+----------+------+------------+

In [0]:
from pyspark.sql.functions import ntile
df.withColumn("ntile",ntile(2).over(windowSpec)) \
    .show()

+-------------+----------+------+-----+
employee_name|department|salary|ntile|
+-------------+----------+------+-----+
 Maria| Finance| 3000| 1|
 Scott| Finance| 3300| 1|
 Jen| Finance| 3900| 2|
 Kumar| Marketing| 2000| 1|
 Jeff| Marketing| 3000| 2|
 James| Sales| 3000| 1|
 James| Sales| 3000| 1|
 Robert| Sales| 4100| 1|
 Saif| Sales| 4100| 2|
 Michael| Sales| 4600| 2|
+-------------+----------+------+-----+

In [0]:
from pyspark.sql.functions import cume_dist    
df.withColumn("cume_dist",cume_dist().over(windowSpec)) \
   .show()

+-------------+----------+------+------------------+
employee_name|department|salary| cume_dist|
+-------------+----------+------+------------------+
 Maria| Finance| 3000|0.3333333333333333|
 Scott| Finance| 3300|0.6666666666666666|
 Jen| Finance| 3900| 1.0|
 Kumar| Marketing| 2000| 0.5|
 Jeff| Marketing| 3000| 1.0|
 James| Sales| 3000| 0.4|
 James| Sales| 3000| 0.4|
 Robert| Sales| 4100| 0.8|
 Saif| Sales| 4100| 0.8|
 Michael| Sales| 4600| 1.0|
+-------------+----------+------+------------------+

In [0]:
from pyspark.sql.functions import lag    
df.withColumn("lag",lag("salary",2).over(windowSpec)) \
      .show()

+-------------+----------+------+----+
employee_name|department|salary| lag|
+-------------+----------+------+----+
 Maria| Finance| 3000|null|
 Scott| Finance| 3300|null|
 Jen| Finance| 3900|3000|
 Kumar| Marketing| 2000|null|
 Jeff| Marketing| 3000|null|
 James| Sales| 3000|null|
 James| Sales| 3000|null|
 Robert| Sales| 4100|3000|
 Saif| Sales| 4100|3000|
 Michael| Sales| 4600|4100|
+-------------+----------+------+----+

In [0]:
from pyspark.sql.functions import lead    
df.withColumn("lead",lead("salary",2).over(windowSpec)) \
    .show()

+-------------+----------+------+----+
employee_name|department|salary|lead|
+-------------+----------+------+----+
 Maria| Finance| 3000|3900|
 Scott| Finance| 3300|null|
 Jen| Finance| 3900|null|
 Kumar| Marketing| 2000|null|
 Jeff| Marketing| 3000|null|
 James| Sales| 3000|4100|
 James| Sales| 3000|4100|
 Robert| Sales| 4100|4600|
 Saif| Sales| 4100|null|
 Michael| Sales| 4600|null|
+-------------+----------+------+----+

In [0]:
windowSpecAgg  = Window.partitionBy("department")
from pyspark.sql.functions import col,avg,sum,min,max,row_number 

df.withColumn("row",row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("department","avg","sum","min","max") \
  .show()


+----------+------+-----+----+----+
department| avg| sum| min| max|
+----------+------+-----+----+----+
 Finance|3400.0|10200|3000|3900|
 Marketing|2500.0| 5000|2000|3000|
 Sales|3760.0|18800|3000|4600|
+----------+------+-----+----+----+

In [0]:
data = [("James","","Smith","36636","M",60000),
        ("Michael","Rose","","40288","M",70000),
        ("Robert","","Williams","42114","",400000),
        ("Maria","Anne","Jones","39192","F",500000),
        ("Jen","Mary","Brown","","F",0)]

columns = ["first_name","middle_name","last_name","dob","gender","salary"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

root
-- first_name: string (nullable = true)
-- middle_name: string (nullable = true)
-- last_name: string (nullable = true)
-- dob: string (nullable = true)
-- gender: string (nullable = true)
-- salary: long (nullable = true)

+----------+-----------+---------+-----+------+------+
first_name|middle_name|last_name|dob |gender|salary|
+----------+-----------+---------+-----+------+------+
James | |Smith |36636|M |60000 |
Michael |Rose | |40288|M |70000 |
Robert | |Williams |42114| |400000|
Maria |Anne |Jones |39192|F |500000|
Jen |Mary |Brown | |F |0 |
+----------+-----------+---------+-----+------+------+

In [0]:
# Using when otherwise
from pyspark.sql.functions import col, when
df2 = df.withColumn("new_gender", when(col("gender") == "M","Male")
                                 .when(col("gender") == "F","Female")
                                 .otherwise("Unknown"))
df2.show(truncate=False)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1489946428265218> in <module> 
 1 # Using when otherwise 
 2 from pyspark . sql . functions import col , when
 ----> 3 df2 = df.withColumn("new_gender", when(col("gender") == "M","Male")
 4 . when ( col ( "gender" ) == "F" , "Female" ) 
 5 .otherwise("Unknown"))

 /databricks/spark/python/pyspark/sql/dataframe.py in withColumn (self, colName, col) 
 2476 """
 2477 assert isinstance ( col , Column ) , "col should be Column" 
 -> 2478 return DataFrame ( self . _jdf . withColumn ( colName , col . _jc ) , self . sql_ctx ) 
 2479 
 2480 def withColumnRenamed ( self , existing , new ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : cannot resolve '`gender`' given input columns: [Amount, Country, Product];
'Project [Product#603, Amount#604L, Country#605, CASE WHEN ('gender = M) THEN Male WHEN ('gender = F) THEN Female ELSE Unknown END AS new_gender#828]
+- LogicalRDD [Product#603, Amount#604L, Country#605], false

In [0]:
df22=df.select(col("*"), when(col("gender") == "M","Male")
      .when(col("gender") == "F","Female")
      .otherwise("Unknown").alias("new_gender")).show(truncate=False)

# Using case when
from pyspark.sql.functions import expr
df3 = df.withColumn("new_gender", expr("case when gender = 'M' then 'Male' " + 
                       "when gender = 'F' then 'Female' " +
                       "else 'Unknown' end"))
df3.show(truncate=False)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1489946428265219> in <module> 
 ----> 1 df22=df.select(col("*"), when(col("gender") == "M","Male")
 2 . when ( col ( "gender" ) == "F" , "Female" ) 
 3 .otherwise("Unknown").alias("new_gender")).show(truncate=False)
 4 
 5 # Using case when 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1690 [ Row ( name = 'Alice' , age = 12 ) , Row ( name = 'Bob' , age = 15 ) ] 
 1691 """
 -> 1692 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1693 return DataFrame ( jdf , self . sql_ctx ) 
 1694 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : cannot resolve '`gender`' given input columns: [Amount, Country, Product];
'Project [Product#603, Amount#604L, Country#605, CASE WHEN ('gender = M) THEN Male WHEN ('gender = F) THEN Female ELSE Unknown END AS new_gender#829]
+- LogicalRDD [Product#603, Amount#604L, Country#605], false

In [0]:
#Using case when
df4 = df.select(col("*"), expr("case when gender = 'M' then 'Male' " +
                       "when gender = 'F' then 'Female' " +
                       "else 'Unknown' end").alias("new_gender"))
df4.show(truncate=False)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1489946428265220> in <module> 
 1 #Using case when 
 ----> 2 df4 = df.select(col("*"), expr("case when gender = 'M' then 'Male' " +
 3 "when gender = 'F' then 'Female' " + 
 4 "else 'Unknown' end").alias("new_gender"))
 5 df4 . show ( truncate = False ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1690 [ Row ( name = 'Alice' , age = 12 ) , Row ( name = 'Bob' , age = 15 ) ] 
 1691 """
 -> 1692 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1693 return DataFrame ( jdf , self . sql_ctx ) 
 1694 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : cannot resolve '`gender`' given input columns: [Amount, Country, Product]; line 1 pos 10;
'Project [Product#603, Amount#604L, Country#605, CASE WHEN ('gender = M) THEN Male WHEN ('gender = F) THEN Female ELSE Unknown END AS new_gender#821]
+- LogicalRDD [Product#603, Amount#604L, Country#605], false

In [0]:
data2 = [(66, "a", "4"), (67, "a", "0"), (70, "b", "4"), (71, "d", "4")]
df5 = spark.createDataFrame(data = data2, schema = ["id", "code", "amt"])
         

df5.withColumn("new_column", when(col("code") == "a" | col("code") == "d", "A")
      .when(col("code") == "b" & col("amt") == "4", "B")
      .otherwise("A1")).show()

--------------------------------------------------------------------------- 
 Py4JError Traceback (most recent call last)
 <command-1489946428265221> in <module> 
 3 
 4 
 ----> 5 df5.withColumn("new_column", when(col("code") == "a" | col("code") == "d", "A")
 6 . when ( col ( "code" ) == "b" & col ( "amt" ) == "4" , "B" ) 
 7 .otherwise("A1")).show()

 /databricks/spark/python/pyspark/sql/column.py in _ (self, other) 
 110 def _ ( self , other ) : 
 111 jc = other . _jc if isinstance ( other , Column ) else other
 --> 112 njc = getattr ( self . _jc , name ) ( jc ) 
 113 return Column ( njc ) 
 114 _ . __doc__ = doc

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 328 format(target_id, ".", name), value)
 329 else : 
 --> 330 raise Py4JError(
 331 "An error occurred while calling {0}{1}{2}. Trace:\n{3}\n" . 
 332 format(target_id, ".", name, value))

 Py4JError : An error occurred while calling o1061.or. Trace:
py4j.Py4JException: Method or([class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:341)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:349)
	at py4j.Gateway.invoke(Gateway.java:286)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
from pyspark.sql.functions import expr

data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

columns= ["Product","Amount","Country"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

root
-- Product: string (nullable = true)
-- Amount: long (nullable = true)
-- Country: string (nullable = true)

+-------+------+-------+
Product|Amount|Country|
+-------+------+-------+
Banana |1000 |USA |
Carrots|1500 |USA |
Beans |1600 |USA |
Orange |2000 |USA |
Orange |2000 |USA |
Banana |400 |China |
Carrots|1200 |China |
Beans |1500 |China |
Orange |4000 |China |
Banana |2000 |Canada |
Carrots|2000 |Canada |
Beans |2000 |Mexico |
+-------+------+-------+

In [0]:
pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.printSchema()
pivotDF.show(truncate=False)

root
-- Product: string (nullable = true)
-- Canada: long (nullable = true)
-- China: long (nullable = true)
-- Mexico: long (nullable = true)
-- USA: long (nullable = true)

+-------+------+-----+------+----+
Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
Orange |null |4000 |null |4000|
Beans |null |1500 |2000 |1600|
Banana |2000 |400 |null |1000|
Carrots|2000 |1200 |null |1500|
+-------+------+-----+------+----+

In [0]:
pivotDF = df.groupBy("Product","Country") \
      .sum("Amount") \
      .groupBy("Product") \
      .pivot("Country") \
      .sum("sum(Amount)")
pivotDF.printSchema()
pivotDF.show(truncate=False)

root
-- Product: string (nullable = true)
-- Canada: long (nullable = true)
-- China: long (nullable = true)
-- Mexico: long (nullable = true)
-- USA: long (nullable = true)

+-------+------+-----+------+----+
Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
Orange |null |4000 |null |4000|
Beans |null |1500 |2000 |1600|
Banana |2000 |400 |null |1000|
Carrots|2000 |1200 |null |1500|
+-------+------+-----+------+----+

In [0]:
""" unpivot """
unpivotExpr = "stack(3, 'Canada', Canada, 'China', China, 'Mexico', Mexico) as (Country,Total)"
unPivotDF = pivotDF.select("Product", expr(unpivotExpr)) \
    .where("Total is not null")
unPivotDF.show(truncate=False)

+-------+-------+-----+
Product|Country|Total|
+-------+-------+-----+
Orange |China |4000 |
Beans |China |1500 |
Beans |Mexico |2000 |
Banana |Canada |2000 |
Banana |China |400 |
Carrots|Canada |2000 |
Carrots|China |1200 |
+-------+-------+-----+

In [0]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.master("local").getOrCreate()

data = [
    ('Thin', 'Cell phone', 6000),
    ('Normal', 'Tablet', 1500),
    ('Mini', 'Tablet', 5500),
    ('Ultra thin', 'Cell phone', 5000),
    ('Vey thin', 'Cell phone', 6000),
    ('Big', 'Tablet', 2500),
    ('Bendable', 'Cell phone', 3000),
    ('Foldable', 'Cell phone', 3000),
    ('Pro', 'Tablet', 5400),
    ('Pro2', 'Tablet', 6500)
]

products = spark.createDataFrame(data, ['product', 'category', 'revenue'])

products.show()

+----------+----------+-------+
 product| category|revenue|
+----------+----------+-------+
 Thin|Cell phone| 6000|
 Normal| Tablet| 1500|
 Mini| Tablet| 5500|
Ultra thin|Cell phone| 5000|
 Vey thin|Cell phone| 6000|
 Big| Tablet| 2500|
 Bendable|Cell phone| 3000|
 Foldable|Cell phone| 3000|
 Pro| Tablet| 5400|
 Pro2| Tablet| 6500|
+----------+----------+-------+

In [0]:
Ответьте на следующие вопросы:

1) Какой продукт является самым продаваемым в каждой категории?
2) Каковы наибольшие и вторые наибольшие по продажам продукты в каждой категории?
3) Найдите разницу между доходом от каждого продукта и самым продаваемым продуктом в той же категории продукта?
4) Найдите разницу между доходом каждого продукта и средним доходом категории, если этот продукт?

In [0]:
products.groupBy("category") \
    .agg(F.max("revenue").alias("best")).show(truncate=False)

+----------+----+
category |best|
+----------+----+
Cell phone|6000|
Tablet |6500|
+----------+----+

In [0]:
from pyspark.sql.window import Window
windowSpec  = Window.partitionBy("category").orderBy("revenue")

products.withColumn("rank", F.rank().over(windowSpec)).where(F.col("rank") == 1)  \
    .show(truncate=False)

+--------+----------+-------+----+
product |category |revenue|rank|
+--------+----------+-------+----+
Bendable|Cell phone|3000 |1 |
Foldable|Cell phone|3000 |1 |
Normal |Tablet |1500 |1 |
+--------+----------+-------+----+

In [0]:
products.withColumn("row_number", F.row_number().over(windowSpec)).where(F.col("row_number") <= 2).drop("row_number")  \
    .show(truncate=False)

+--------+----------+-------+
product |category |revenue|
+--------+----------+-------+
Bendable|Cell phone|3000 |
Foldable|Cell phone|3000 |
Normal |Tablet |1500 |
Big |Tablet |2500 |
+--------+----------+-------+

In [0]:
drop(columns=)
select("nam1", "name2")